In [1]:
import tensorflow as tf
import os, sys, pickle, math, warnings, time
import pandas as pd
import numpy as np
import tensorflow_probability as tfp
from sklearn.preprocessing import StandardScaler
# 宏定义
tfd = tfp.distributions
dtype = np.float32
tfe = tf.contrib.eager
try:
    tfe.enable_eager_execution()
except ValueError:
    pass

In [7]:
help(tf.get_variable)

Help on function get_variable in module tensorflow.python.ops.variable_scope:

get_variable(name, shape=None, dtype=None, initializer=None, regularizer=None, trainable=None, collections=None, caching_device=None, partitioner=None, validate_shape=True, use_resource=None, custom_getter=None, constraint=None, synchronization=<VariableSynchronization.AUTO: 0>, aggregation=<VariableAggregation.NONE: 0>)
    Gets an existing variable with these parameters or create a new one.
    
    This function prefixes the name with the current variable scope
    and performs reuse checks. See the
    [Variable Scope How To](https://tensorflow.org/guide/variables)
    for an extensive description of how reusing works. Here is a basic example:
    
    ```python
    def foo():
      with tf.variable_scope("foo", reuse=tf.AUTO_REUSE):
        v = tf.get_variable("v", [1])
      return v
    
    v1 = foo()  # Creates v.
    v2 = foo()  # Gets the same, existing v.
    assert v1 == v2
    ```
    
    If i

In [6]:
df = pd.read_csv('../winequality-white.csv', sep=';')
df.head()
X = df[features].values
X.shape

(4898, 11)

In [1]:
import tensorflow as tf

state = tf.Variable(0, name='counter')

# 定义常量 one
one = tf.constant(1)

# 定义加法步骤 (注: 此步并没有直接计算)
new_value = tf.add(state, one)

# 将 State 更新成 new_value
update = tf.assign(state, new_value)

In [2]:
# 如果定义 Variable, 就一定要 initialize
# init = tf.initialize_all_variables() # tf 马上就要废弃这种写法
init = tf.global_variables_initializer()  # 替换成这样就好
 
# 使用 Session
with tf.Session() as sess:
    sess.run(init)
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

1
2
3


In [9]:

n = tfd.Normal(loc=0., scale=1.)
n

<tf.distributions.Normal 'Normal_1/' batch_shape=() event_shape=() dtype=float32>

In [47]:



fs = np.array([1, 2, 3])
p = [.2, .3, .5]
dist = tfd.Multinomial(total_count=3., probs=p)
samples = dist.sample(1)[0]
with tf.Session() as sess:
    samples = sess.run(samples)
samples = samples.astype(np.int32)
print(samples)
fs_s = []
for i in range(3):
    while(samples[i]!=0):
        fs_s.append(fs[i])
        samples[i] -= 1

#fs_s = fs[samples]
#with tf.Session() as sess:
    #sample = sess.run(samples)
#    out  = sess.run(fs_s)
print(fs_s)


[0 2 1]
[2, 2, 3]


In [11]:
import tensorflow as tf

a = tf.constant(1)
b = tf.constant(2)
c = tf.constant(3)

def cond(a, b, c):
    return a<5

def body(a, b, c):
    #a += 1
    b += 1
    c +=1
    return a+1, b+1, c # same with [a, b, c]

a ,b, c = tf.while_loop(cond, body, [a,b,c]) 

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(a)) #[5, 6, 7]

5


In [13]:
def make_likelihood(true_variances):
    return tfd.MultivariateNormalDiag(
      scale_diag=tf.sqrt(true_variances))

dims = 10
dtype = np.float32
true_variances = tf.linspace(dtype(1), dtype(3), dims)
likelihood = make_likelihood(true_variances)

states, kernel_results = tfp.mcmc.sample_chain(
    num_results=1000,
    current_state=tf.zeros(dims),
    kernel=tfp.mcmc.HamiltonianMonteCarlo(
      target_log_prob_fn=likelihood.log_prob,
      step_size=0.5,
      num_leapfrog_steps=2),
    num_burnin_steps=500)

# Compute sample stats.
sample_mean = tf.reduce_mean(states, axis=0)
sample_var = tf.reduce_mean(
    tf.squared_difference(states, sample_mean),
    axis=0)

In [22]:
with tf.Session() as sess:
    sts = sess.run(states)
st = sts[-1]

In [23]:
print(sts[-1])
print(st)

[-0.11028031  0.51820797  0.90102303  1.1792982   1.1975937   0.48935413
 -1.0380324   3.461598    2.0996425  -1.9550304 ]
[-0.11028031  0.51820797  0.90102303  1.1792982   1.1975937   0.48935413
 -1.0380324   3.461598    2.0996425  -1.9550304 ]


#### 计算v

#### 对于Theta的重采样
待完成...

#### MH链-SMC第三步
只取返回结果的最后一轮作为样本

```
tfp.mcmc.sample_chain(
    num_results, # K
    current_state, # samples whose leftmost dimesion's B
    previous_kernel_results=None,
    kernel=None, 
    num_burnin_steps=0,
    num_steps_between_results=0,
    parallel_iterations=10,
    name=None
)
```


In [2]:
target = tfd.Normal(loc=dtype(0), scale=dtype(1))

samples, _ = tfp.mcmc.sample_chain(
  num_results=1000,
  current_state=dtype(1),
  kernel=tfp.mcmc.RandomWalkMetropolis(
     target.log_prob,
     seed=42),
  num_burnin_steps=500,
  parallel_iterations=1)  # For determinism.

sample_mean = tf.reduce_mean(samples, axis=0)
sample_std = tf.sqrt(
    tf.reduce_mean(tf.squared_difference(samples, sample_mean),
                   axis=0))
with tf.Session() as sess:
  [sample_mean_, sample_std_] = sess.run([sample_mean, sample_std])

print('Estimated mean: {}'.format(sample_mean_))
print('Estimated standard deviation: {}'.format(sample_std_))

Estimated mean: -0.10047363489866257
Estimated standard deviation: 0.9807838201522827


In [11]:
# SMC参数设置
J = 10    #迭代轮数
B = 10**4 #并行样本数
K = 16    #每次MH的轮数

layer_size = [8, 7, 10, 1]
l2_norm = 0.0003

In [35]:
# 数据导入以及标准化处理
features = ['fixed acidity','volatile acidity','residual sugar','chlorides','free sulfur dioxide','density','pH','sulphates']
df = pd.read_csv('../winequality-white.csv', sep=';')
X = df[features].values
y = (df.quality >= 7).values

X = StandardScaler().fit_transform(X).astype(dtype)

In [54]:
def init_theta():
    theta_W = []
    theta_b = []
    i = 0
    for s1,s2 in zip(layer_size[:-1], layer_size[1:]):
        i = i+1
        print(i)
        theta_W.append(np.random.normal(0, 10, [s1,s2]).astype(dtype))
        theta_b.append(np.random.normal(0, 10, [s2]).astype(dtype))
    return theta_W, theta_b

In [41]:
t1 = 0
for w in theta_W:
        t = w * w
        print(t.shape)
        t1 += np.sum(t)

(8, 7)
(7, 10)
(10, 1)


In [31]:
a = tf.multiply(3, np.eye(4))
with tf.Session() as sess:
    a = sess.run(a)
    print(a)

[[3 0 0 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 0 0 3]]


In [42]:
def Ln(theta_W, theta_b):
    # 预测
#    st = time.time()
    pred = X
    for w,b in zip(theta_W[:-1],theta_b[:-1]):
        pred = tf.nn.relu(np.matmul(pred, w) + b)
    pred = -tfnn.sigmoid_cross_entropy_with_logits(
        labels=y, 
        logits=(tf.matmul(pred, theta_W[-1]) + theta_b[-1]))
    return tf.reduce_sum(pred) #n is not defined here so use sum

In [30]:
v = np.exp((phi[j]-phi[j-1]) * n * np.array(list(map(lambda x:Ln(x), samples))))

0

In [55]:
B = 10
Ws = []
bs = []
for i in range(B):
    tW, tb = init_theta()
    Ws.append(tW)
    bs.append(tb)

1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3


In [52]:
for i in range(B):
    print(Ws[i], bs[i])
print(len(Ws), len(bs))

[array([[  2.8782682 ,  -2.596048  ,  -5.606595  ,   8.615315  ,
         -1.3102044 ,   8.48805   , -23.140255  ],
       [ 17.520407  ,  10.767909  , -12.283993  ,   8.980462  ,
        -21.695553  ,   4.8747764 ,  10.693069  ],
       [ 11.225842  ,   8.360376  ,   0.8933277 ,   9.738124  ,
          1.5607617 ,  23.170918  ,  10.308323  ],
       [ 20.977816  ,  -2.1933732 , -11.98423   ,   7.0745206 ,
          9.566854  ,  -9.664131  , -28.988626  ],
       [ -1.5837424 , -20.709785  ,  -5.402558  ,   4.738639  ,
         13.769042  ,   0.7852478 ,   4.1791754 ],
       [ -1.026469  ,   2.50755   , -15.286637  ,  -8.01364   ,
          4.6750684 , -10.351065  ,  -6.62151   ],
       [  2.7422373 ,  -1.6724775 ,  11.714129  , -12.317547  ,
         13.328377  ,   2.7046907 , -10.861852  ],
       [  1.7599941 ,  -0.28446993,  -0.4316553 ,   9.185539  ,
         -7.307872  ,   3.9594085 ,  -8.895451  ]], dtype=float32), array([[  3.5338087 ,  21.024693  ,   6.791007  ,   7.8236876 

In [53]:
ln = tf.map_fn(fn=lambda x:Ln(x[0], x[1]),
              elems=(Ws, bs))

TypeError: Object arrays are not currently supported

In [57]:
help(tf.map_fn)

Help on function map_fn in module tensorflow.python.ops.functional_ops:

map_fn(fn, elems, dtype=None, parallel_iterations=10, back_prop=True, swap_memory=False, infer_shape=True, name=None)
    map on the list of tensors unpacked from `elems` on dimension 0.
    
    The simplest version of `map_fn` repeatedly applies the callable `fn` to a
    sequence of elements from first to last. The elements are made of the
    tensors unpacked from `elems`. `dtype` is the data type of the return
    value of `fn`. Users must provide `dtype` if it is different from
    the data type of `elems`.
    
    Suppose that `elems` is unpacked into `values`, a list of tensors. The shape
    of the result tensor is `[values.shape[0]] + fn(values[0]).shape`.
    
    This method also allows multi-arity `elems` and output of `fn`.  If `elems`
    is a (possibly nested) list or tuple of tensors, then each of these tensors
    must have a matching first (unpack) dimension.  The signature of `fn` may
    matc